<a href="https://colab.research.google.com/github/junecv/EAD/blob/main/Openai_Gym_Pacman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Environment



In [ ]:
'''
Credit: Ryan Rudes https://github.com/ryanrudes/colabgymrender for video rendering wrapper
Due to bug fix, I included all source code below with fixes on env.render()
Comment out for install / import until pull request accepted.
'''

# !pip install colabgymrender
# from colabgymrender.recorder import Recorder

In [1]:
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install imageio==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 5.2 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303885 sha256=e5efd4cc02f24d680d38c2243e786a01f74e133437175c1b604563eb567b98f3
  Stored in directory: /root/.cache/pip/wheels/46/20/07/7bb9c8c44e6ec2efa60fd0e6280094f53f65f41767ef69a5ee
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfully uninstalled imageio-2.9.0


In [2]:
# Restart runtime following the instruction after installs

!apt-get update > /dev/null 2>&1
# !apt-get install cmake > /dev/null 2>&1
# !pip install --upgrade setuptools 2>&1
# !pip install ez_setup > /dev/null 2>&1
!pip install -U gym > =0.21.0
!pip install -U gym[atari,accept-rom-license] > /dev/null 2>&1

### Imports


In [4]:
import os
import time
import gym
import cv2

from moviepy.editor import *
from IPython.core.display import Video, display
from IPython.terminal.interactiveshell import TerminalInteractiveShell

### Pacman

In [7]:
'''
Credit: Ryan Rudes https://github.com/ryanrudes/colabgymrender
With fixes on env.render() arg. due to TypeError
'''

class Recorder(gym.Wrapper):
    def __init__(self, env, directory, auto_release=True, size=None, fps=30, rgb=True):   # changed to default fps=30 due to fps alert
        super(Recorder, self).__init__(env)
        self.directory = directory
        self.auto_release = auto_release
        self.active = True
        self.rgb = rgb

        if not os.path.exists(self.directory):
            os.mkdir(self.directory)

        if size is None:
            self.env.reset()
            # self.size = self.env.render(mode = 'rgb_array').shape[:2][::-1]   # comment out original 
            self.size = self.env.render().shape[:2][::-1]
        else:
            self.size = size

        if fps is None:
            if 'video.frames_per_second' in self.env.metadata:
                self.fps = self.env.metadata['video.frames_per_second']
            else:
                self.fps = 30
        else:
            self.fps = fps

    def pause(self):
        self.active = False

    def resume(self):
        self.active = True

    def _start(self):
        self.cliptime = time.time()
        self.path = f'{self.directory}/{self.cliptime}.mp4'
        fourcc = cv2.VideoWriter_fourcc(*'MP4V')
        self._writer = cv2.VideoWriter(self.path, fourcc, self.fps, self.size)

    def _write(self):
        if self.active:
            # frame = self.env.render(mode = 'rgb_array')                       # comment out original 
            frame = self.env.render()
            if self.rgb:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            self._writer.write(frame)

    def release(self):
        self._writer.release()

    def reset(self, *args, **kwargs):
        observation = self.env.reset(*args, **kwargs)
        self._start()
        self._write()
        return observation

    def step(self, *args, **kwargs):
        data = self.env.step(*args, **kwargs)
        self._write()

        if self.auto_release and data[2]:
            self.release()

        return data

    def play(self):
        start = time.time()
        filename = 'temp-{start}.mp4'
        clip = VideoFileClip(self.path)
        clip.write_videofile(filename, progress_bar = False, verbose = False)
        display(Video(filename, embed = True))
        os.remove(filename)

In [8]:
env = gym.make("ALE/MsPacman-v5", render_mode = 'rgb_array')
directory = './video'
env = Recorder(env, directory)

observation = env.reset()
terminal = False
while not terminal:
  action = env.action_space.sample()
  observation, reward, terminal, truncated, info = env.step(action)

env.play()